## Circuit_tool_ai_bot

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_CIRCUIT_API_KEY"))  # Load your OpenAI API key from environment variables

# Example form data
form_data = {
    "Name": "John Doe",
    "Service Number": "123456",
    "Claim Type": "Disability Compensation",
    "Details": "Veteran served in the U.S. Army from 2000 to 2005 and sustained injuries during service."
}

def generate_summary(form_data):
    prompt = f"Create a summary report for the following VA form data:\n{form_data}"
    try:
        # Call OpenAI's GPT model
        response = client.chat.completions.create(model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a summerizer assistant that only summerizes VA forms."},
            {"role": "user", "content": prompt}
        ],
        )
        # Extract and return the assistant's response
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
summary_report = generate_summary(form_data)
print("Summary Report:")
print(summary_report)

Summary Report:
Summary Report:

- Name: John Doe
- Service Number: 123456
- Claim Type: Disability Compensation
- Details: Veteran served in the U.S. Army from 2000 to 2005 and sustained injuries during service.


In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import re
import spacy

# Ensure the spaCy model is installed
try:
    spacy.load("en_core_web_sm")
except OSError:
    # Install the model if not found
    !python -m spacy download en_core_web_sm
from base64 import b64encode

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_CIRCUIT_API_KEY"))

# Load spaCy for NLP (question detection)
nlp = spacy.load("en_core_web_sm")

def extract_text_from_image(image_path):
    """Extract text from image using OpenAI Vision API."""
    try:
        with open(image_path, "rb") as image_file:
            image_data = b64encode(image_file.read()).decode("utf-8")
        
        response = client.chat.completions.create(
            model="gpt-4-vision-preview",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Extract text describing circuit parts (e.g., MOTOR, BOLT, O RING, CIRCUIT BREAKER) and their attributes (e.g., voltage, diameter, amps). Ignore irrelevant text."
                        },
                        {"type": "image_url", "image_url": f"data:image/jpeg;base64,{image_data}"}
                    ]
                }
            ],
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error processing image: {e}"

def is_question(text):
    """Detect if input is a question."""
    doc = nlp(text.lower())
    return any(token.text in ["what", "how", "why", "tell"] or token.tag_ == "WP" for token in doc)

def answer_question(text):
    """Provide general info for part-related questions."""
    text = text.lower()
    if "motor" in text:
        return ("A motor converts electrical energy to mechanical motion. Key specs include voltage (V), "
                "current (A), power (kW/HP), frequency (Hz), and phase. Can you share details about your motor?")
    elif "o ring" in text or "o-ring" in text:
        return ("An O-ring is a seal to prevent leaks, defined by outer diameter (OD), inner diameter (ID), "
                "and thickness (THK), e.g., 14MM OD X 10MM ID X 3MM THK. Do you have a specific O-ring?")
    elif "circuit breaker" in text:
        return ("A circuit breaker protects circuits from overloads, rated by amps, voltage, and poles, "
                "e.g., 10A, 400V, 2 POLE. Do you have a specific circuit breaker?")
    elif "bolt" in text:
        return ("A bolt is a fastener with a threaded shank, defined by type, diameter, and length, "
                "e.g., HEX, 2 IN DIA X 6 5/8 IN LG. Do you have a specific bolt?")
    return "I’m not sure what part you’re asking about. Can you clarify or provide an image/text?"

def parse_part(line):
    """Parse a single part line into formatted output."""
    line = line.strip().upper()
    
    # Regex patterns for parts
    motor_pattern = r"MOTOR,?\s*([\d.-/]+)?\s*(V|A|KW|HP|HZ|RPM|IP\d+|TYPE\s*[A-Z]|\d\s*PHASE)?"
    breaker_pattern = r"CIRCUIT BREAKER,?\s*([\d]+)\s*A,?\s*([\d/-]+)?\s*(V)?,?\s*(\d)?\s*(POLE)?"
    bolt_pattern = r"BOLT,?\s*([A-Z]+)?,?\s*([\d\s/]+)\s*IN\s*DIA\s*X\s*([\d\s/]+)?\s*IN\s*LG"
    oring_pattern = r"O RING,?\s*(\w+)?,?\s*([\d]+)\s*MM\s*OD\s*X\s*([\d]+)\s*MM\s*ID\s*X\s*([\d\s/]+)\s*MM\s*THK"
    
    if "MOTOR" in line:
        attributes = re.findall(motor_pattern, line)
        attrs = {v.split()[1]: v.split()[0] for v in attributes if v[0]}
        formatted = [
            attrs.get("A", "AMPS UNKNOWN"),
            attrs.get("V", "VOLTS UNKNOWN"),
            attrs.get("KW", attrs.get("HP", "POWER UNKNOWN")),
            attrs.get("HZ", "HZ UNKNOWN"),
            attrs.get("PHASE", "3 PHASE"),
            attrs.get("IP", ""),
            attrs.get("RPM", ""),
            attrs.get("TYPE", "")
        ]
        formatted = [x for x in formatted if x]
        prompt = ("I see a motor. Can you provide missing details like "
                  f"{'current (amps)' if 'AMPS UNKNOWN' in formatted else ''} "
                  f"{'power (kW/HP)' if 'POWER UNKNOWN' in formatted else ''} "
                  f"{'frequency (Hz)' if 'HZ UNKNOWN' in formatted else ''}?").strip()
        return f"MOTOR, {', '.join(formatted)}", prompt
    
    elif "CIRCUIT BREAKER" in line:
        match = re.match(breaker_pattern, line)
        if match:
            volts = f"{match.group(2)}V" if match.group(2) else "VOLTS UNKNOWN"
            poles = f"{match.group(4)} POLE" if match.group(4) else "POLES UNKNOWN"
            prompt = ("Can you provide " +
                      f"{'voltage' if volts == 'VOLTS UNKNOWN' else ''} " +
                      f"{'poles' if poles == 'POLES UNKNOWN' else ''}?").strip()
            return f"CIRCUIT BREAKER, {match.group(1)}A, {volts}, {poles}", prompt
    
    elif "BOLT" in line:
        match = re.match(bolt_pattern, line)
        if match:
            length = match.group(3) or "LENGTH UNKNOWN"
            prompt = "Can you provide the bolt’s length?" if length == "LENGTH UNKNOWN" else ""
            return f"BOLT, {match.group(1) or ''}, {match.group(2)} IN DIA X {length}".strip(", "), prompt
    
    elif "O RING" in line:
        match = re.match(oring_pattern, line)
        if match:
            material = match.group(1) or ""
            return f"O RING, {material}, {match.group(2)}MM OD X {match.group(3)}MM ID X {match.group(4)}MM THK".strip(", "), ""
    
    return f"UNRECOGNIZED: {line}", "I don’t recognize this part. Can you provide more details or an image?"

def process_input(image_path=None, text=None):
    """Process image or text input and return formatted parts list with prompts."""
    try:
        if image_path:
            text = extract_text_from_image(image_path)
        if not text:
            return [("No input provided", "Please upload an image or provide text describing a part.")]
        
        lines = text.split("\n")
        results = []
        for line in lines:
            line = line.strip()
            if not line:
                continue
            if is_question(line):
                results.append((answer_question(line), ""))
            else:
                part, prompt = parse_part(line)
                results.append((part, prompt))
        return results
    except Exception as e:
        return [(f"Error processing input: {e}", "Please try again or provide a different image/text.")]

# Example usage
image_input = "images/1000004257.jpg"  # Replace with actual image path
text_input = """
tell me about this
"""

# Test with text input
print("Processing text input:")
# results = process_input(text=text_input)
# for part, prompt in results:
#     print(f"Result: {part}")
#     if prompt:
#         print(f"Prompt: {prompt}")

# Test with image input (uncomment and provide valid image path)
print("\nProcessing image input:")
results = process_input(image_path=image_input)
for part, prompt in results:
    print(f"Result: {part}")
    if prompt:
        print(f"Prompt: {prompt}")

Processing text input:

Processing image input:
Result: UNRECOGNIZED: ERROR PROCESSING IMAGE: 'IMAGES' OBJECT HAS NO ATTRIBUTE 'CREATE'
Prompt: I don’t recognize this part. Can you provide more details or an image?


In [19]:
import base64
import os
from openai import OpenAI
from dotenv import load_dotenv

# Initialize OpenAI client with the API key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_CIRCUIT_API_KEY"))


# Function to encode the image into base64 format
def encode_image(image_path):
    """Encode an image into a base64 string."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Function to process the image and text input
def process_image_and_text(image_path, question):
    """Process image and provide text-based response from OpenAI API."""
    if not os.path.exists(image_path):
        return "Image file not found."

    # Encode the image to base64
    base64_image = encode_image(image_path)

    try:
        # Send the image data and question to OpenAI for analysis
        response = client.chat.completions.create(
            model="gpt-4.1-mini",  # Use the correct model, for example: gpt-4
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a marine engineering assistant specializing in ship propulsion systems, with expertise strictly limited to diesel and steam engines used in maritime vessels. ",
                        "When answering questions, always reference specific components from these engines, such as motors, bolts, bearings, O-rings, etc., and explain their roles in the propulsion system. You have detailed knowledge of these components, including their specifications and standard description formats, such as 'MOTOR, 440V, 92A, 56.5KW, 60HZ, 3 PHASE' and 'BOLT, HEX, 2 IN DIA X 6 5/8 IN LG'.", 
                        "Maintain a professional and informative tone, avoiding unnecessary jargon to ensure clarity for users with varying levels of expertise. If a question falls outside your domain, politely inform the user and suggest they ask about diesel or steam engine propulsion systems instead."
                    ),
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": question},  # The question
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            },
                        },  # Base64 image
                    ],
                },
            ],
            max_tokens=500,  # Adjust token limit as needed
        )

        # Retrieve and return the result from OpenAI's response
        # return response['choices'][0]['message']['content']
        return response.choices[0].message.content

    except Exception as e:
        return f"Error processing image: {e}"


# Example usage:
image_path = "images/1000002410.jpg"  # Replace with actual image path
question = "What's in this image?"  # Example question

# Process image and get the response
result = process_image_and_text(image_path, question)
print(result)

This image shows a set of large, curved exhaust manifolds typically used in the exhaust system of a marine diesel engine. These manifolds collect exhaust gases from multiple cylinders and direct them into a single outlet. The metal surfaces display signs of heat exposure, which is common for exhaust parts. The labels with barcodes and the tags likely indicate part numbers or maintenance information for identification and tracking.

In a ship's propulsion system, these exhaust manifolds are crucial components as they safely route high-temperature exhaust gases away from the engine to the exhaust stack, preventing heat damage and enabling efficient engine operation.


In [8]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import re
import spacy
import pdfplumber
from base64 import b64encode
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_CIRCUIT_API_KEY"))

# Load spaCy for NLP
nlp = spacy.load("en_core_web_sm")

# Simulated knowledge base (replace with FAISS/Pinecone for production)
knowledge_base = {
    "MOTOR, 92A, 440V, 56.5KW, 60HZ, 3 PHASE, IP54, 3565 RPM": None,
    "BOLT, HEX, 2 IN DIA X 6 5/8 IN LG": None,
    "O RING, RUBBER, 14MM OD X 10MM ID X 3MM THK": None,
    "CIRCUIT BREAKER, 10A, 400V, 2 POLE": None
}

def get_text_embedding(text):
    """Generate embedding for text using OpenAI."""
    try:
        response = client.embeddings.create(model="text-embedding-ada-002", input=text)
        return np.array(response.data[0].embedding)
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return None

def initialize_knowledge_base():
    """Embed knowledge base entries."""
    for text in knowledge_base:
        embedding = get_text_embedding(text)
        knowledge_base[text] = embedding

def extract_text_from_image(image_data):
    """Extract text from image using OpenAI Vision API."""
    try:
        response = client.chat.completions.create(
            model="gpt-4-vision-preview",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Extract text describing circuit parts (e.g., MOTOR, BOLT) and attributes (e.g., voltage, diameter). Ignore irrelevant text."
                        },
                        {"type": "image_url", "image_url": f"data:image/jpeg;base64,{image_data}"}
                    ]
                }
            ],
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error processing image: {e}"

def extract_pdf_content(pdf_path):
    """Extract text and images from PDF."""
    text = ""
    image_texts = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                # Extract text
                text += page.extract_text() or ""
                # Extract images
                for img in page.images:
                    img_data = img["stream"].get_data()
                    img_base64 = b64encode(img_data).decode("utf-8")
                    img_text = extract_text_from_image(img_base64)
                    image_texts.append(img_text)
        return text, image_texts
    except Exception as e:
        return f"Error processing PDF: {e}", []

def is_question(text):
    """Detect if input is a question."""
    doc = nlp(text.lower())
    return any(token.text in ["what", "how", "why", "tell"] or token.tag_ == "WP" for token in doc)

def answer_question(text):
    """Provide general info for part-related questions."""
    text = text.lower()
    if "motor" in text:
        return ("A motor converts electrical energy to mechanical motion. Key specs include voltage (V), "
                "current (A), power (kW/HP), frequency (Hz), and phase. Can you share details about your motor?")
    elif "o ring" in text or "o-ring" in text:
        return ("An O-ring is a seal to prevent leaks, defined by outer diameter (OD), inner diameter (ID), "
                "and thickness (THK), e.g., 14MM OD X 10MM ID X 3MM THK. Do you have a specific O-ring?")
    elif "circuit breaker" in text:
        return ("A circuit breaker protects circuits from overloads, rated by amps, voltage, and poles, "
                "e.g., 10A, 400V, 2 POLE. Do you have a specific circuit breaker?")
    elif "bolt" in text:
        return ("A bolt is a fastener with a threaded shank, defined by type, diameter, and length, "
                "e.g., HEX, 2 IN DIA X 6 5/8 IN LG. Do you have a specific bolt?")
    return "I’m not sure what part you’re asking about. Can you clarify or provide an image/text?"

def parse_part(line):
    """Parse a single part line into formatted output."""
    line = line.strip().upper()
    
    # Regex patterns
    motor_pattern = r"MOTOR,?\s*([\d.-/]+)?\s*(V|A|KW|HP|HZ|RPM|IP\d+|TYPE\s*[A-Z]|\d\s*PHASE)?"
    breaker_pattern = r"CIRCUIT BREAKER,?\s*([\d]+)\s*A,?\s*([\d/-]+)?\s*(V)?,?\s*(\d)?\s*(POLE)?"
    bolt_pattern = r"BOLT,?\s*([A-Z]+)?,?\s*([\d\s/]+)\s*IN\s*DIA\s*X\s*([\d\s/]+)?\s*IN\s*LG"
    oring_pattern = r"O RING,?\s*(\w+)?,?\s*([\d]+)\s*MM\s*OD\s*X\s*([\d]+)\s*MM\s*ID\s*X\s*([\d\s/]+)\s*MM\s*THK"
    
    if "MOTOR" in line:
        attributes = re.findall(motor_pattern, line)
        attrs = {v.split()[1]: v.split()[0] for v in attributes if v[0]}
        formatted = [
            attrs.get("A", "AMPS UNKNOWN"),
            attrs.get("V", "VOLTS UNKNOWN"),
            attrs.get("KW", attrs.get("HP", "POWER UNKNOWN")),
            attrs.get("HZ", "HZ UNKNOWN"),
            attrs.get("PHASE", "3 PHASE"),
            attrs.get("IP", ""),
            attrs.get("RPM", ""),
            attrs.get("TYPE", "")
        ]
        formatted = [x for x in formatted if x]
        prompt = ("I see a motor. Can you provide missing details like "
                  f"{'current (amps)' if 'AMPS UNKNOWN' in formatted else ''} "
                  f"{'power (kW/HP)' if 'POWER UNKNOWN' in formatted else ''} "
                  f"{'frequency (Hz)' if 'HZ UNKNOWN' in formatted else ''}?").strip()
        return f"MOTOR, {', '.join(formatted)}", prompt
    
    elif "CIRCUIT BREAKER" in line:
        match = re.match(breaker_pattern, line)
        if match:
            volts = f"{match.group(2)}V" if match.group(2) else "VOLTS UNKNOWN"
            poles = f"{match.group(4)} POLE" if match.group(4) else "POLES UNKNOWN"
            prompt = ("Can you provide " +
                      f"{'voltage' if volts == 'VOLTS UNKNOWN' else ''} " +
                      f"{'poles' if poles == 'POLES UNKNOWN' else ''}?").strip()
            return f"CIRCUIT BREAKER, {match.group(1)}A, {volts}, {poles}", prompt
    
    elif "BOLT" in line:
        match = re.match(bolt_pattern, line)
        if match:
            length = match.group(3) or "LENGTH UNKNOWN"
            prompt = "Can you provide the bolt’s length?" if length == "LENGTH UNKNOWN" else ""
            return f"BOLT, {match.group(1) or ''}, {match.group(2)} IN DIA X {length}".strip(", "), prompt
    
    elif "O RING" in line:
        match = re.match(oring_pattern, line)
        if match:
            material = match.group(1) or ""
            return f"O RING, {material}, {match.group(2)}MM OD X {match.group(3)}MM ID X {match.group(4)}MM THK".strip(", "), ""
    
    return f"UNRECOGNIZED: {line}", "I don’t recognize this part. Can you provide more details or an image?"

def find_similar_parts(text):
    """Find similar parts in knowledge base using semantic similarity."""
    input_embedding = get_text_embedding(text)
    if input_embedding is None:
        return None
    similarities = []
    for kb_text, kb_embedding in knowledge_base.items():
        if kb_embedding is not None:
            sim = cosine_similarity([input_embedding], [kb_embedding])[0][0]
            similarities.append((kb_text, sim))
    if similarities:
        return max(similarities, key=lambda x: x[1])[0]  # Return most similar part
    return None

def process_input(image_path=None, text=None, pdf_path=None):
    """Process image, text, or PDF input and return formatted parts list with prompts."""
    try:
        all_text = []
        if pdf_path:
            pdf_text, image_texts = extract_pdf_content(pdf_path)
            if isinstance(pdf_text, str) and pdf_text.startswith("Error"):
                return [(pdf_text, "Please try a different PDF.")]
            all_text.append(pdf_text)
            all_text.extend(image_texts)
        elif image_path:
            with open(image_path, "rb") as image_file:
                image_data = b64encode(image_file.read()).decode("utf-8")
            image_text = extract_text_from_image(image_data)
            if image_text.startswith("Error"):
                return [(image_text, "Please upload a clearer image.")]
            all_text.append(image_text)
        elif text:
            all_text.append(text)
        else:
            return [("No input provided", "Please upload an image, PDF, or provide text describing a part.")]

        results = []
        for text_chunk in all_text:
            lines = text_chunk.split("\n")
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                if is_question(line):
                    results.append((answer_question(line), ""))
                else:
                    part, prompt = parse_part(line)
                    # Check for similar parts in knowledge base
                    similar_part = find_similar_parts(part)
                    if similar_part and similar_part != part:
                        prompt += f" This part is similar to: {similar_part}"
                    results.append((part, prompt))
        return results
    except Exception as e:
        return [(f"Error processing input: {e}", "Please try again or provide a different input.")]

# Initialize knowledge base embeddings
initialize_knowledge_base()

# Example usage
text_input = """
MOTOR, 200-230V, TYPE P
What’s an O-ring?
BOLT, 2 IN DIA, HEX
"""

pdf_input = "circuit_parts.pdf"  # Replace with actual PDF path
image_input = "circuit_image.jpg"  # Replace with actual image path

# Test with text input
print("Processing text input:")
results = process_input(text=text_input)
for part, prompt in results:
    print(f"Result: {part}")
    if prompt:
        print(f"Prompt: {prompt}")

# Test with PDF input (uncomment and provide valid PDF path)
# print("\nProcessing PDF input:")
# results = process_input(pdf_path=pdf_input)
# for part, prompt in results:
#     print(f"Result: {part}")
#     if prompt:
#         print(f"Prompt: {prompt}")

ModuleNotFoundError: No module named 'pdfplumber'